In [1]:
!git clone https://github.com/sooo66/semeval2026-task12-dataset.git

Cloning into 'semeval2026-task12-dataset'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 69 (delta 30), reused 51 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (69/69), 6.72 MiB | 19.88 MiB/s, done.
Resolving deltas: 100% (30/30), done.


In [2]:
import json
import re

def clean_text(text):
    """
    پاک‌سازی اولیه متون
    """
    if not text or not isinstance(text, str):
        return ""
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def load_and_merge_data(questions_path, docs_path):
    # ۱. خواندن فایل اسناد (docs.json)
    with open(docs_path, 'r', encoding='utf-8') as f:
        docs_raw = json.load(f)
        
    # ۲. تبدیل هوشمند لیست به دیکشنری برای جستجوی سریع بر اساس topic_id
    docs_dict = {}
    
    # بررسی می‌کنیم که آیا فایل اصلی لیستی از آیتم‌هاست
    if isinstance(docs_raw, list):
        for item in docs_raw:
            if not isinstance(item, dict):
                continue
                
            t_id = item.get("topic_id")
            if not t_id:
                continue
                
            if t_id not in docs_dict:
                docs_dict[t_id] = []
                
            # حالت اول: اگر آیتم شامل لیستی از اسناد در کلید docs باشد
            if "docs" in item or "documents" in item:
                inner_docs = item.get("docs", item.get("documents", []))
                for d in inner_docs:
                    if isinstance(d, dict):
                        # استخراج متن از دیکشنری داخلی
                        text_val = d.get("text", d.get("content", d.get("doc", "")))
                        docs_dict[t_id].append(text_val)
                    elif isinstance(d, str):
                        docs_dict[t_id].append(d)
            # حالت دوم: اگر خود آیتم مستقیماً یک سند باشد
            else:
                text_val = item.get("text", item.get("content", item.get("doc", "")))
                if text_val:
                    docs_dict[t_id].append(text_val)

    merged_dataset = []
    
    # ۳. خواندن فایل سوالات و اتصال به اسناد
    with open(questions_path, 'r', encoding='utf-8') as f:
        for line in f:
            if not line.strip():
                continue
                
            question_item = json.loads(line.strip())
            topic_id = question_item.get("topic_id")
            
            # استخراج اسناد از دیکشنری که بالاتر ساختیم (این بار بدون خطا)
            related_docs = docs_dict.get(topic_id, [])
            
            # پاک‌سازی و ترکیب متون
            cleaned_docs = [clean_text(doc) for doc in related_docs if doc]
            context_text = " ".join(cleaned_docs)
            
            # ۴. ساختار نهایی
            merged_item = {
                "id": question_item.get("id"),
                "topic_id": topic_id,
                "target_event": clean_text(question_item.get("target_event")),
                "options": {
                    "A": clean_text(question_item.get("option_A")),
                    "B": clean_text(question_item.get("option_B")),
                    "C": clean_text(question_item.get("option_C")),
                    "D": clean_text(question_item.get("option_D"))
                },
                "golden_answer": question_item.get("golden_answer"),
                "context": context_text 
            }
            merged_dataset.append(merged_item)
            
    return merged_dataset

# ==========================================
# اجرای کد
# ==========================================

QUESTIONS_FILE = 'semeval2026-task12-dataset/sample_data/questions.jsonl'
DOCS_FILE = 'semeval2026-task12-dataset/sample_data/docs.json'

dataset = load_and_merge_data(QUESTIONS_FILE, DOCS_FILE)

if dataset:
    print("✅ پردازش با موفقیت انجام شد!\n")
    print(f"🔹 رویداد هدف: {dataset[0]['target_event']}")
    print(f"🔹 گزینه‌ها:")
    for key, val in dataset[0]['options'].items():
        print(f"   {key}: {val}")
    print(f"🔹 پاسخ درست: {dataset[0]['golden_answer']}")
    print(f"🔹 طول متن زمینه (Context): {len(dataset[0]['context'])} کاراکتر")
else:
    print("مجموعه داده خالی است!")

✅ پردازش با موفقیت انجام شد!

🔹 رویداد هدف: Videos of the assassination circulated on social media.
🔹 گزینه‌ها:
   A: The shooter used a handmade gun.
   B: Security arrested the suspected gunman, Tetsuya Yamagami.
   C: Shinzo Abe became the deputy chief cabinet secretary in the early 2000s.
   D: A man fired twice at Shinzo Abe.
🔹 پاسخ درست: D
🔹 طول متن زمینه (Context): 94617 کاراکتر


In [5]:
# در صورت نیاز کتابخانه‌های زیر را نصب کنید:
# !pip install transformers torch scikit-learn sentence-transformers

import json
import re
import torch
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# ==========================================
# ۰. بارگذاری مدل تعبیه‌سازی (Embedding) برای جستجوی معنایی
# ==========================================
print("در حال بارگذاری مدل Embedding (برای RAG معنایی)...")
# یک مدل سبک، سریع و بسیار قدرتمند برای تبدیل متن به بردار مفهومی
embedder = SentenceTransformer('all-MiniLM-L6-v2') 

# ==========================================
# ۱. تابع بازیابی اطلاعات (Semantic RAG) برای فیلتر کانتکست
# ==========================================
def retrieve_relevant_context(sample, top_k=5):
    """
    این تابع جملات کانتکست را بر اساس شباهت معنایی (Semantic Similarity) با کوئری استخراج می‌کند.
    """
    context = sample['context']
    
    # شکستن متن طولانی به جملات مجزا
    sentences = re.split(r'(?<=[.!?])\s+', context)
    sentences = [s.strip() for s in sentences if len(s.strip()) > 20]
    
    if not sentences:
        return ""

    # ساخت کوئری: رویداد هدف + تمام گزینه‌ها
    query = sample['target_event'] + " " + " ".join(sample['options'].values())

    # تبدیل جملات و کوئری به بردارهای معنایی
    sentence_embeddings = embedder.encode(sentences)
    query_embedding = embedder.encode([query])

    # محاسبه شباهت کسینوسی بین بردارها
    similarities = cosine_similarity(query_embedding, sentence_embeddings).flatten()
    
    # استخراج جملات برتر و حفظ ترتیب زمانی آن‌ها
    top_indices = similarities.argsort()[-top_k:]
    top_indices_sorted = sorted(top_indices) 
    
    best_sentences = [sentences[i] for i in top_indices_sorted]
    return " ".join(best_sentences)

# ==========================================
# ۲. تابع ساخت پیام‌ها بر اساس Chain of Thought (CoT)
# ==========================================
def generate_messages(sample, filtered_context):
    """
    در این پرامپت، مدل تشویق می‌شود ابتدا استدلال کند و سپس جواب را در تگ <ANSWER> قرار دهد.
    """
    messages = [
        {
            "role": "system", 
            "content": "You are an expert AI specialized in Abductive Event Reasoning. First, briefly reason about the direct cause. Then, you MUST output the uppercase letter(s) of the correct option(s) inside XML tags like this: <ANSWER>A</ANSWER> or <ANSWER>A,C</ANSWER>. Do not output anything else after the tags."
        },
        {
            "role": "user", 
            "content": f"""Context:
{filtered_context}

Target Event: {sample['target_event']}

Options:
A) {sample['options']['A']}
B) {sample['options']['B']}
C) {sample['options']['C']}
D) {sample['options']['D']}

Which option(s) is/are the most likely direct cause of the Target Event? Provide a brief reasoning, then wrap your final answer in <ANSWER> tags."""
        }
    ]
    return messages

# ==========================================
# ۳. شبیه‌سازی فراخوانی مدل (LLM Inference)
# ==========================================
print("\nدر حال بارگذاری مدل زبانی اصلی...")
pipe = pipeline(
    "text-generation", 
    model="Qwen/Qwen2.5-7B-Instruct", 
    device_map="auto",
    torch_dtype=torch.float16 
)
print("✅ مدل‌ها با موفقیت بارگذاری شدند.\n")

# فرض می‌کنیم متغیر dataset از اجرای گام‌های قبلی موجود است
sample_data = dataset[0]

smart_context = retrieve_relevant_context(sample_data, top_k=5)
messages = generate_messages(sample_data, smart_context)
formatted_prompt = pipe.tokenizer.apply_chat_template(
    messages, 
    tokenize=False, 
    add_generation_prompt=True
)

print("\nدر حال تولید پاسخ (با استدلال)...")
response = pipe(
    formatted_prompt, 
    max_new_tokens=200, # افزایش فضا برای نوشتن استدلال
    temperature=0.1,    # دمای پایین برای منطق استوار
    do_sample=True,
    pad_token_id=pipe.tokenizer.eos_token_id
)

generated_text = response[0]['generated_text']
raw_answer = generated_text.replace(formatted_prompt, "").strip()

# ت) استخراج هوشمندانه از داخل تگ <ANSWER>
match = re.search(r'<ANSWER>\s*([A-D](?:\s*,\s*[A-D])*)\s*</ANSWER>', raw_answer)
if match:
    clean_answer = match.group(1).replace(' ', '')
else:
    # تلاش مجدد (Fallback) اگر مدل تگ‌ها را رعایت نکرد
    fallback_match = re.search(r'[A-D](?:\s*,\s*[A-D])*', raw_answer)
    clean_answer = fallback_match.group(0).replace(' ', '') if fallback_match else "نامشخص"

print("-" * 50)
print(f"🧠 استدلال کامل مدل:\n{raw_answer}")
print("-" * 50)
print(f"✨ خروجی نهایی و استخراج شده: {clean_answer}")
print(f"🎯 پاسخ درست (Golden Answer): {sample_data['golden_answer']}")

در حال بارگذاری مدل Embedding (برای RAG معنایی)...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

✅ مدل‌ها با موفقیت بارگذاری شدند.



Passing `generation_config` together with generation-related arguments=({'temperature', 'max_new_tokens', 'do_sample', 'pad_token_id'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



در حال تولید پاسخ (با استدلال)...
--------------------------------------------------
🧠 استدلال کامل مدل:
The most likely direct cause of the videos of the assassination circulating on social media is that someone recorded the event and shared it online. This action is a result of witnessing the event or obtaining footage from another source.

<ANSWER>B</ANSWER>
--------------------------------------------------
✨ خروجی نهایی و استخراج شده: B
🎯 پاسخ درست (Golden Answer): D


In [6]:
import json
import re
from tqdm import tqdm

# ==========================================
# تابع ارزیابی
# ==========================================
def calculate_score(predicted_answers_str, golden_answers_str):
    if predicted_answers_str == "نامشخص":
        return 0.0
        
    P = set([ans.strip() for ans in predicted_answers_str.split(',') if ans.strip()])
    G = set([ans.strip() for ans in golden_answers_str.split(',') if ans.strip()])
    
    if P == G and len(P) > 0:
        return 1.0  
    elif P.issubset(G) and len(P) > 0:
        return 0.5  
    else:
        return 0.0  

# ==========================================
# ارزیابی کلی روی کل دیتاست (با Semantic RAG و CoT)
# ==========================================
print(f"\nشروع ارزیابی روی {len(dataset)} نمونه...")
total_score = 0
results_log = []

for index, sample in enumerate(tqdm(dataset, desc="Processing Samples")):
    
    # ۱. استخراج کانتکست معنایی
    smart_context = retrieve_relevant_context(sample, top_k=5)
    
    # ۲. ساخت پرامپت
    messages = generate_messages(sample, smart_context)
    formatted_prompt = pipe.tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    
    # ۳. تولید خروجی (با فضای بیشتر برای استدلال)
    response = pipe(
        formatted_prompt, 
        max_new_tokens=200, 
        temperature=0.1, 
        do_sample=True,
        pad_token_id=pipe.tokenizer.eos_token_id
    )
    
    generated_text = response[0]['generated_text']
    raw_answer = generated_text.replace(formatted_prompt, "").strip()
    
    # ۴. استخراج از تگ ANSWER
    match = re.search(r'<ANSWER>\s*([A-D](?:\s*,\s*[A-D])*)\s*</ANSWER>', raw_answer)
    if match:
        clean_answer = match.group(1).replace(' ', '')
    else:
        fallback_match = re.search(r'[A-D](?:\s*,\s*[A-D])*', raw_answer)
        clean_answer = fallback_match.group(0).replace(' ', '') if fallback_match else "نامشخص"
    
    # ۵. محاسبه امتیاز
    golden = sample['golden_answer']
    score = calculate_score(clean_answer, golden)
    total_score += score
    
    results_log.append({
        "id": sample['id'],
        "target_event": sample['target_event'],
        "predicted": clean_answer,
        "golden": golden,
        "score": score
    })

# ==========================================
# نمایش نتایج نهایی
# ==========================================
average_score = total_score / len(dataset)
print("\n" + "="*50)
print("🏆 نتایج نهایی ارزیابی (Semantic RAG + CoT) 🏆")
print("="*50)
print(f"تعداد کل نمونه‌ها: {len(dataset)}")
print(f"مجموع امتیازات کسب شده: {total_score}")
print(f"میانگین امتیاز (Accuracy): {average_score:.4f} یا {average_score * 100:.2f}%")
print("="*50)

print("\n🔍 تحلیل خطاها (نمونه‌های با امتیاز ۰):")
error_count = 0
for res in results_log:
    if res['score'] == 0.0:
        print(f"ID: {res['id']} | Predicted: {res['predicted']} | Golden: {res['golden']}")
        error_count += 1
        if error_count >= 5:
            break


شروع ارزیابی روی 200 نمونه...



Processing Samples:   0%|          | 0/200 [00:00<?, ?it/s]Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)

Processing Samples:   0%|          | 1/200 [00:03<12:32,  3.78s/it]Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)

Processing Samples:   1%|          | 2/200 [00:10<17:50,  5.41s/it]Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)

Processing Samples:   2%|▏         | 3/200 [00:17<20:27,


🏆 نتایج نهایی ارزیابی (Semantic RAG + CoT) 🏆
تعداد کل نمونه‌ها: 200
مجموع امتیازات کسب شده: 122.5
میانگین امتیاز (Accuracy): 0.6125 یا 61.25%

🔍 تحلیل خطاها (نمونه‌های با امتیاز ۰):
ID: q-6 | Predicted: D | Golden: C
ID: q-9 | Predicted: B | Golden: D
ID: q-10 | Predicted: B | Golden: A,C
ID: q-12 | Predicted: C | Golden: A
ID: q-16 | Predicted: C | Golden: B
